### Data Preprocessing
We removed all the stop words, punctuations and normalized the texts to lower case. 


### Model
We will use Naive Bayes as our baseline model. 

Parametrization:

We let $x_j$ denote the identity of the j-th word in a text response. Thus, xj is now an integer taking values in {1, . . . , |V|}, where |V| is the size of our vocabulary (dictionary). A sample of $d$ words is now represented by a vector $(x_1, x_2, . . . , x_d)$ of length d.

Naive Bayes assumes that $p(x_j|y)$ are independent from each other. That $p(x_1,...x_d|y) = p(x_1|y)p(x_2|y)...p(x_d|y)$. We also asume that $x_j|y$ comes from a multinomial distribution, with parameters 
$\phi_{k|y=1} (p(x_j = k\,|\,y=1))$ and $\phi_{k|y=0} (p(x_j = k\,|\,y=0))$

Then the likehood of the data is given by 

$$L(\phi_y,\phi_{k|y=1},\phi_{k|y=0}) = \prod_{i=1}^np(x^i,y^i) = \prod_{i=1}^n \big(\prod_{j=1}^{d^i} p(x_j^{i}|y;\phi_{k|y=0},\phi_{k|y=1})\big)p(y^i;\phi_y)$$


We find the MLE estimates of the parameters to fit the model.

To find the most indicative words for treatment/control, we use the metric:

$$log\bigg(\frac{P(word\_i \,|\,treatment)}{P(word\_i\,|\,control)}\bigg)$$


In [1]:
import pandas as pd
import numpy as np

In [2]:
# read the entire survey data
df = pd.read_csv("data/study_data.csv")
df.info()
df.groupby(['arm']).size()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 5 columns):
arm                470 non-null object
pre_q_original     470 non-null object
post_q_original    470 non-null object
pre_q              470 non-null object
post_q             470 non-null object
dtypes: object(5)
memory usage: 18.4+ KB


arm
arch_graphic    94
arch_strong     94
arch_weak       94
control         94
curb_appeal     94
dtype: int64

In [3]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    words = message.split()
    normalized_words = list(map(lambda word:word.lower(),words))
    return normalized_words
    # *** END CODE HERE ***


def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    from collections import defaultdict
    sentence_freq = defaultdict(int)
    count = 0
    word_int = {}
    for i in range(len(messages)):
        unique= defaultdict(int)

        # get the unique words in the messages
        for word in get_words(messages[i]):
            unique[word]+=1
        
        # add the words to dictionary only if they appear in at least five messages
        for unique_word in unique:
            sentence_freq[unique_word]+=1
            if sentence_freq[unique_word] ==9:
                word_int[unique_word] = count
                count+=1

    return word_int

    # *** END CODE HERE ***


def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    of the vocabulary appears in each message. 
    Each row in the resulting array should correspond to each message 
    and each column should correspond to a word of the vocabulary.

    Use the provided word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
        Where the component (i,j) is the number of occurrences of the
        j-th vocabulary word in the i-th message.
    """
    # *** START CODE HERE ***
    table = np.zeros((len(messages),len(word_dictionary)))
    all_words = list(word_dictionary.keys())

    for i in range(len(messages)):
        words = get_words(messages[i])
        for j in range(len(word_dictionary)):
            for word in words:
                if word == all_words[j]:    
                    table[i,j] +=1

    return table


In [15]:
def fit_naive_bayes_model(matrix, labels, treatment_label, control_label):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    n_words = matrix.shape[1]

    treatment = matrix[labels == treatment_label,:]
    control = matrix[labels == control_label,:]

    length_treatment = np.sum(treatment) #number of words in all spam texts
    length_control = np.sum(control) #number of words in all non-spam texts

    model = {}
    model['prob_treatment'] = (np.sum(treatment,axis=0)+1)/(length_treatment+n_words)
    model['prob_control'] = (np.sum(control,axis=0)+1)/(length_control+n_words)
    model['phi'] = treatment.shape[0]/(treatment.shape[0]+control.shape[0])

    return model

    # *** END CODE HERE ***


def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    log_prob_treatment = np.sum(np.log(model['prob_treatment'])*matrix,axis=1)
    log_prob_control = np.sum(np.log(model['prob_control'])*matrix,axis=1)
    phi = model['phi']
    
    prob = 1 / (1 + np.exp(log_prob_control + np.log(1-phi) - log_prob_treatment - np.log(phi)))
    pred = np.zeros(len(prob))
    pred[np.argwhere(prob>0.5)]=1
    return pred
    # *** END CODE HERE ***


def get_top_naive_bayes_words(model, dictionary,num):
    """Compute the top five words that are most indicative of the treatment (i.e positive) class.

    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: A list of the top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***
    log_treatment = np.log(model['prob_treatment'])
    log_control = np.log(model['prob_control'])

    to_sort = list(zip(log_treatment-log_control,dictionary.keys()))

    # take first element for sort
    def takeFirst(elem):
        return elem[0]
    
    to_sort.sort(key=takeFirst,reverse=True)
    top_five_words = list(zip(*to_sort))[1][0:num]
    return top_five_words


    # *** END CODE HERE ***

In [16]:
import re
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = re.sub(r'[,!@#$%^&*)(|/><";:.?\'\\}{-]',"",text)
    text = text.lower()
    return text

In [17]:
def train_nb (data, treatment_label,control_label,top_num):
    
    # pre-process data
    treatment_control = df[df['arm'].isin([treatment_label,control_label])]
    train_responses = np.array(treatment_control['post_q'].apply(cleanText))
    train_labels = treatment_control['arm']
    word_dict = create_dictionary(train_responses)
    train_matrix = transform_text(train_responses,word_dict)
    
    # fit naive bayes and get training accuracy
    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels, treatment_label,control_label)
    naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, train_matrix)
    
    binary_train_labels = train_labels.copy()
    binary_train_labels[train_labels == treatment_label] = 1
    binary_train_labels[train_labels == control_label] = 0

    naive_bayes_accuracy = np.mean(naive_bayes_predictions == binary_train_labels)
    top_words = get_top_naive_bayes_words(naive_bayes_model, word_dict,top_num)
    
    return naive_bayes_accuracy, top_words

    

In [18]:
# arch strong vs control
acc, top_words = train_nb (df, 'arch_strong','control',15)
print("Training accuracy is", acc, " \n Top indicative words are ",top_words)

Training accuracy is 0.8563829787234043  
 Top indicative words are  ('georgian', 'symmetrical', 'multi', 'bay', 'pane', 'gabled', 'symmetry', 'also', 'over', 'design', 'feature', 'center', '5', 'door', 'stone')


In [56]:
# arch weak and control
acc, top_words = train_nb (df, 'arch_weak','control',15)
print("Training accuracy is", acc, " \n Top indicative words are ",top_words)

Training accuracy is 0.8191489361702128  
 Top indicative words are  ('georgian', 'triangular', 'decorative', 'symmetric', 'symmetrical', 'multi', 'paned', 'typical', 'center', 'open', 'pane', '5', 'style', 'stone', 'across')


In [57]:
# arch strong and arch weak
acc, top_words = train_nb (df, 'arch_strong','arch_weak',15)
print("Training accuracy is", acc, " \n Top indicative words are ",top_words)

Training accuracy is 0.8297872340425532  
 Top indicative words are  ('bay', 'gabled', 'dormer', 'colonial', 'four', 'facade', 'over', 'appear', '12', 'pane', 'floor', 'design', 'symmetry', 'seem', 'place')


In [58]:
# curb appeal and control
acc, top_words = train_nb (df, 'curb_appeal','control',15)
print("Training accuracy is", acc, " \n Top indicative words are ",top_words)

Training accuracy is 0.7606382978723404  
 Top indicative words are  ('curb', 'way', 'appeal', 'dull', 'can', 'entry', 'doesnt', 'do', 'flower', 'drab', 'exterior', 'plant', 'would', 'color', 'classic')


In [59]:
acc, top_words = train_nb (df, 'curb_appeal','arch_strong',15)
print("Training accuracy is", acc, " \n Top indicative words are ",top_words)

Training accuracy is 0.8297872340425532  
 Top indicative words are  ('curb', 'appeal', 'but', 'way', 'dull', 'much', 'lot', 'natural', 'landscape', 'frame', 'color', 'plant', 'welcome', 'not', 'you')


Next we will try the Bernoulli Naive Bayes model. It might outperform the multinomial event.

In [73]:

import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

treatment_label = 'arch_strong'
control_label = 'arch_weak'
treatment_control = df[df['arm'].isin([treatment_label,control_label])]
X = np.array(treatment_control['post_q'].apply(cleanText))
y = treatment_control['arm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [71]:
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=['treatment','control']))

accuracy 0.5263157894736842
             precision    recall  f1-score   support

  treatment       0.52      0.58      0.55        19
    control       0.53      0.47      0.50        19

avg / total       0.53      0.53      0.53        38



Bernoulli NB actually outperforms multinomial NB. Therefore we will use Bernoulli NB as our baseline.

In [72]:
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print  (coef_1, fn_1, coef_2, fn_2)

        
clf = nb.named_steps['clf']        
vect = nb.named_steps['vect']
show_most_informative_features(vect,clf)

-7.026686477312418 1800s -4.439835923799467 the
-7.026686477312418 1880s -4.565165086412526 be
-7.026686477312418 1940s -4.678453607717966 it
-7.026686477312418 3rd -4.983291179932513 house
-7.026686477312418 accentuate -5.033777745688296 have
-7.026686477312418 adjoin -5.099620697480413 of
-7.026686477312418 after -5.124033886222779 and
-7.026686477312418 again -5.14688726437217 window
-7.026686477312418 against -5.22601289569945 with
-7.026686477312418 alabaster -5.275981774420433 this
-7.026686477312418 align -5.276618697887858 in
-7.026686477312418 alone -5.359216156053652 style
-7.026686477312418 along -5.387432325418917 look
-7.026686477312418 amazingly -5.393444339280163 roof
-7.026686477312418 any -5.44848325276888 home
-7.026686477312418 apart -5.45521509479523 front
-7.026686477312418 apparent -5.504643033847124 its
-7.026686477312418 appeal -5.506961491133849 on
-7.026686477312418 appearance -5.511048216871162 georgian
-7.026686477312418 architectural -5.534294996028426 tria

(1, 755)